## WTI Oil Prices
* Lag features
#### Economic growth
* US dollar index 
* S&P 500 index 
* US Federal Reserve rate
* Volatility index (VIX)
* Consumer price index (CPI).

source: https://savearchive.zbw.eu/bitstream/11159/5075/1/1747444766.pdf

#### Weather data
source: https://www.eia.gov/finance/markets/crudeoil/spot_prices.php
#### Supply and Demand
* OPEC

source: https://www.investopedia.com/articles/investing/072515/top-factors-reports-affect-price-oil.asp


## Energy Information Administration (EIA) Api
| Features | Frequence | Data Until |
|---|---|---|
| WTI | Daily | up-to-date |
| Oil Production | Monthly | July 2025|
|Weekly Input Utilization | Weekly | up-to-date |
| Gasoline Price | Weekly | up-to-date |
|Imports & Exports | Weekly | up-to-date |
|Crude Oil Supplied | Monthly | July 2025 |

In [3]:
import os
os.chdir('..')

In [4]:
%pwd

'c:\\Users\\User\\Desktop\\DS\\Projects\\oil-optimization'

In [2]:
from oil_optimization import logger

In [5]:
import pandas as pd
import requests
from dotenv import dotenv_values

secrets = dotenv_values('.env')

api_calls  = {
    'wti_prices':{
        'url':'https://api.eia.gov/v2/petroleum/pri/spt/data',
        'payload':{
                    'api_key':secrets['EIA_API_KEY'],
                    'facets[product][]':'EPCWTI',
                    'start':'2015-01-01',
                    'sort[0][column]':'period',
                    'sort[0][direction]':'asc',
                    'frequency':'daily',
                    'data[0]':'value'
                }
    },
    'oil_production':{
        'url':'https://api.eia.gov/v2/petroleum/crd/crpdn/data',
        'payload':{
                    'api_key':secrets['EIA_API_KEY'],
                    'facets[product][]':'EPC0',
                    'start':'2015-01-01',
                    'sort[0][column]':'period',
                    'sort[0][direction]':'asc',
                    'data[0]':'value'
                },
        'date_intervals':[
                    ('2015-01','2020-02'),
                    ('2020-02','2025-03'),
                    ('2025-03','')
                    ]
    },
    'input_utilization':{
        'url':'https://api.eia.gov/v2/petroleum/pnp/wiup/data',
        'payload':{
                    'api_key':secrets['EIA_API_KEY'],
                    'facets[product][]':'EPC0',
                    'start':'2015-01-01',
                    'sort[0][column]':'period',
                    'sort[0][direction]':'asc',
                    'data[0]':'value'
                }
    },
    'gasoline_price':{
        'url':'https://api.eia.gov/v2/petroleum/pri/gnd/data',
        'payload':{
                    'api_key':secrets['EIA_API_KEY'],
                    'facets[product][]':'EPM0',
                    'start':'2015-01-01',
                    'sort[0][column]':'period',
                    'sort[0][direction]':'asc',
                    'data[0]':'value',
                },
        'date_intervals':[
                    ('2015-01-01','2017-12-31'),
                    ('2018-01-01','2021-03-31'),
                    ('2021-04-01','2024-06-30'),
                    ('2024-07-01',None)
                    ]
    },
    'imports_and_exports':{
        'url':'https://api.eia.gov/v2/petroleum/move/wkly/data',
        'payload':{
                    'api_key':secrets['EIA_API_KEY'],
                    'facets[product][]':'EPC0',
                    'start':'2015-01-01',
                    'sort[0][column]':'period',
                    'sort[0][direction]':'asc',
                    'data[0]':'value'
                },
        'date_intervals':[
                    ('2015-01-01','2023-08-31'),
                    ('2023-09-01','')
                    ]
    },
    'oil_supplied':{
        'url':'https://api.eia.gov/v2/petroleum/cons/psup/data',
        'payload':{
                    'api_key':secrets['EIA_API_KEY'],
                    'facets[product][]':'EPC0',
                    'start':'2015-01-01',
                    'sort[0][column]':'period',
                    'sort[0][direction]':'asc',
                    'data[0]':'value'
                }
    }}

In [6]:
import time
file_name = 'oil_production'
params_dict = api_calls[file_name]
# r = requests.get(params_dict['url'], params=params_dict['payload'])

In [ ]:
from src.oil_optimization.utils.io_helpers import read_yaml

config = read_yaml('config/config.yml')
api_config = read_yaml('config/api_config.yml')
eia_api_calls = api_config['eia_api']

data_dir = config['data_ingestion']['root_dir']

In [ ]:
data_list = []
for key,params_dict in eia_api_calls.items():
    

wti_prices {'url': 'https://api.eia.gov/v2/petroleum/pri/spt/data', 'payload': {'api_key': None, 'facets[product][]': 'EPCWTI', 'start': '2015-01-01', 'sort[0][column]': 'period', 'sort[0][direction]': 'asc', 'frequency': 'daily', 'data[0]': 'value'}}
oil_production {'url': 'https://api.eia.gov/v2/petroleum/crd/crpdn/data', 'payload': {'api_key': None, 'facets[product][]': 'EPC0', 'start': '2015-01-01', 'sort[0][column]': 'period', 'sort[0][direction]': 'asc', 'data[0]': 'value'}, 'date_intervals': [['2015-01', '2020-02'], ['2020-02', '2025-03'], ['2025-03', None]]}
input_utilization {'url': 'https://api.eia.gov/v2/petroleum/pnp/wiup/data', 'payload': {'api_key': None, 'facets[product][]': 'EPC0', 'start': '2015-01-01', 'sort[0][column]': 'period', 'sort[0][direction]': 'asc', 'data[0]': 'value'}}
gasoline_price {'url': 'https://api.eia.gov/v2/petroleum/pri/gnd/data', 'payload': {'api_key': None, 'facets[product][]': 'EPM0', 'start': '2015-01-01', 'sort[0][column]': 'period', 'sort[0][

In [13]:
if "date_intervals" in params_dict.keys():
    dates = params_dict['date_intervals']
    for i,date in enumerate(dates):
        
        ##This should go in a function
        params_dict['payload']['start'] = date[0]
        if date[1]:
            params_dict['payload']['end'] = date[1]
        else:
            del params_dict['payload']['end']
        try:
            print(f'Requesting from {date[0]} to {date[1]}')
            r = requests.get(params_dict['url'], params=params_dict['payload'])
            print(f'Status Code: {r.status_code}')
            r.raise_for_status()
            response_data = r.json()['response']['data']
            data_list.append(pd.DataFrame(response_data))
            time.sleep(5)
        except requests.exceptions.HTTPError as e:
            print(f'status code: {r.status_code} | Retrying...')
            for attempt in range(5):
                r = requests.get(params_dict['url'], params=params_dict['payload'],timeout=10)
                time.sleep(5)
                if r.status_code == 200:
                    response_data = r.json()['response']['data']
                    data_list.append(pd.DataFrame(response_data))
                    print("Successful retrial")
                    break
                else:
                    print(f'Retrial {attempt+1}: Failed | Status code: {r.status_code}')


df = pd.concat(data_list,axis=0).reset_index(drop=True)
df.to_csv(f'{data_dir}/raw/{file_name}.csv', index=False)

Requesting from 2015-01 to 2020-02
Status Code: 200
Requesting from 2020-02 to 2025-03
Status Code: 200
Requesting from 2025-03 to 
Status Code: 200


In [9]:
%%writefile src/oil_optimization/data_extractor/eia_extractor.py

from src.oil_optimization.utils.io_helpers import read_yaml, save_csv
import pandas as pd

class EIAExtractor:
    def __init__(self, config_path:str, api_config_path, api:str = 'eia_api') -> None:
        self.config = read_yaml(config_path)
        self.eia_api_config = read_yaml(api_config_path)[api]
        self.data_dir = config['data_ingestion']['root_dir']
        self.dataframes = []

    def _make_request(self, url: str, payload: dict, retries: int = 5, sleep: int = 5):
        for attempt in range(retries):
            try:
                r = requests.get(url, payload)
                print(f'Status Code: {r.status_code}')
                r.raise_for_status()
                return pd.DataFrame(r.json()['response']['data'])
            
            except requests.exceptions.HTTPError as e:
                print(f'status code: {r.status_code} | Retrying...')
                time.sleep(sleep)


    def extract_data(self):
        for key, params_dict in self.eia_api_config.items():
            if "date_intervals" in params_dict.keys():
                for i, date in enumerate(dates):
                    payload = params_dict['payload'].copy()
                    payload['start'] = date[0]
                    if date[1]:
                        payload['end'] = date[1]
                    df = self._make_request(params_dict['url'], payload=payload)

                    self.dataframes.append(df)

    def save_to_csv(self, df: pd.DataFrame, filename: str):
        path = f'data/{filename}.csv'
        save_csv(df=df, path=path)
        print(f'Datafile {filename} saved!')
        

Writing src/oil_optimization/data_extractor/eia_extractor.py


In [30]:
for i in range(10):
    if i%2 == 0:
        continue
    print(i)

1
3
5
7
9


## Stock Market Indexes
* S&P 500 Energy

In [56]:
import yfinance as yf

In [ ]:
energy = yf.download('^GSPE',start='2015-01-01')    

C:\Users\User\AppData\Local\Temp\ipykernel_1752\627084361.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  energy = yf.download('^GSPE',start='2015-01-01')
[*********************100%***********************]  1 of 1 completed


In [78]:
energy.Close.reset_index().to_csv('data/energy.csv',index=False)

In [13]:
import pprint
from src.oil_optimization.utils.io_helpers import read_yaml
content = read_yaml('config/config.yml')
pprint.pprint(content)

{'data_ingestion': {'api_endpoint': ['https://api.eia.gov/v2/crude-oil-imports/data',
                                     'https://api.eia.gov/v2/petroleum/pri/spt/data'],
                    'macro_indicators_urls': ['https://contenido.bce.fin.ec/documentos/informacioneconomica/indicadores/real/PrecioPetroleoCrudoEcuatoriano.html',
                                              'https://contenido.bce.fin.ec/documentos/informacioneconomica/indicadores/real/Inflacion.html'],
                    'oil_reports_url': ['https://contenido.bce.fin.ec/documentos/Estadisticas/Hidrocarburos/SerieCifrasPetroleras.xlsx'],
                    'root_dir': 'data',
                    'yfinance': {'start_date': '2015-01-01',
                                 'ticker': '^GSPE'}}}


## Federeal Reserve Bank of St. Louis (FRED)